<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Linkedin+Notion : Update metrics in content calendar from posts
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/LinkedIn/LinkedIn_Get_posts_stats_from_profile.ipynb" target="_parent"><img src="https://img.shields.io/badge/-Open%20in%20Naas-success?labelColor=000000&logo="/></a>

**Tags:** #linkedin #profile #post #feed #naas_drivers #notion

## Input

### Scheduler your notebook

In [ ]:
import naas
naas.scheduler.add(cron="0 */6 * * *")

### Import libraries

In [ ]:
import naas
from naas_drivers import linkedin, notion, emailbuilder, gsheet
from datetime import datetime
import pandas as pd
import plotly.express as px
import os

### Setup your email

In [ ]:
# Email
EMAIL = "jeremy.ravenel@cashstory.com"

### Get your cookies
<a href='https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75'>How to get your cookies ?</a>

In [ ]:
# Lindekin cookies
LI_AT = naas.secret.get("JRV_LI_AT")
JSESSIONID = naas.secret.get("JRV_JSESSIONID")

### Enter your LinkedIn profile URL
<a href='https://www.linkedin.com/'>=> Go to LinkedIn</a>

In [ ]:
# Linkedin profile url
profile_url = "https://www.linkedin.com/in/j%C3%A9r%C3%A9my-ravenel-8a396910"

### Enter your Notion parameters
<a href='https://www.notion.so/'>=> Go to Notion</a>

In [ ]:
# Notion parameters
notion_token = naas.secret.get("notion_token")
notion_database = 'https://www.notion.so/naas-official/724fec443b134f288b356001bb1543bd?v=c82a8005a5bf4862b7c967a9689aa799'

## Model

### Get content calendar from Notion

In [ ]:
db_notion = notion.connect(notion_token).database.get(notion_database)
df_notion = db_notion.df()
print("📊 Post in Notion database:", len(df_notion))

### Get last post in Notion database

In [ ]:
df_last_post = df_notion.sort_values(by="Publication Date", ascending=False).reset_index(drop=True)
post_url = df_last_post.loc[0, "Content URL"]
print("👉 Last post url:", post_url)
# df_last_post.head(1)

### Get posts not created in notion database

In [ ]:
df_posts_feed = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(profile_url, until={"POST_URL": post_url})
df_posts_feed = df_posts_feed[df_posts_feed.POST_URL != post_url]
print("🚀 New posts:", len(df_posts_feed))
# df_posts_feed

### Get stats from last 50 posts

In [ ]:
df_posts_stats = linkedin.connect(LI_AT, JSESSIONID).profile.get_posts_feed(profile_url, count=50)
print("🔄 Posts stats to update:", len(df_posts_stats))
# df_posts_stats

## Output

In [ ]:
def create_polls_graph(uid, title, data):
    # Create dataframe
    df = pd.melt(pd.DataFrame([data]))
    df = df.sort_values(by="value")
    voters = df.value.sum()
    
    # Create fig
    fig = px.bar(df,
                 y="variable",
                 x="value",
                 orientation='h',
                 title=f"{title}<br><span style='font-size: 13px;'>Total amount of votes: {voters}</span>",
                 text="value",
                 labels={"variable": "Poll options",
                         "value": "Poll results"}
                 )
    fig.update_traces(marker_color='black')
    fig.update_layout(
        plot_bgcolor="#ffffff",
        width=600,
        height=400,
        font=dict(family="Arial", size=14, color="black"),
        paper_bgcolor="white",
        xaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_title=None,
        margin_pad=10,
    )
    fig.write_html(f"{uid}.html")
    asset = naas.asset.add(f"{uid}.html", params={"inline": True})
    return asset

In [ ]:
def update_dynamic_properties(page, row):
    # Page properties : dynamic
    page.number("Engagment score", float(row.ENGAGEMENT_SCORE))
    page.number("Views", int(row.VIEWS))
    page.number("Likes", int(row.LIKES))
    page.number("Comments", int(row.COMMENTS))
    page.number("Shares", int(row.SHARES))

    # Add polls
    poll_urn = row.POLL_URN
    poll_question = row.POLL_QUESTION
    poll_results = row.POLL_RESULTS
    if poll_urn and poll_question and poll_results:
        poll_graph = create_polls_graph(poll_urn, poll_question, poll_results)
        page.embed(poll_graph)
    page.update()
    print(f"✅ Post stats updated in notion for page '{row.TITLE}'.")
    return page

def update_content_notion(df, database_url):
    # Decode database id
    database_id = database_url.split("/")[-1].split("?v=")[0]
    
    # Get pages from notion database
    pages = notion.database.query(database_id, query={})
    
    # Manage dataframe empty
    if len(df) == 0:
        print(f"🛑 Nothing to update in Notion.")
        return
    
    # Loop in data
    df.COMPANY_MENTION = df.COMPANY_MENTION.fillna("")
    df.PROFILE_MENTION = df.PROFILE_MENTION.fillna("")
    for i, row in df.iterrows():
        title = row.TITLE
        post_url = row.POST_URL
        image_url = row.IMAGE_URL
        cover = None
        if image_url is not None:
            cover = image_url
        
        # Create or update page
        page_new = True
        for page in pages:
            page_temp = page.df()
            page_id = page_temp.loc[page_temp.Name == "Content URL", "Value"].values
            if page_id == post_url:
                page_new = False
                break
        try:
            if page_new:
                # Create new page in notion
                page = notion.Page.new(database_id=database_id).create()

                # Page properties : static
                page.date("Publication Date", row.PUBLISHED_DATE)
                page.title("Name", title)
                page.select("Content type", row.CONTENT)
                page.select("Platform", "LinkedIn")
                page.select("Status", "Published ✨")
                page.rich_text("Profile mention", row.PROFILE_MENTION)
                page.rich_text("Company mention", row.COMPANY_MENTION)
                page.number("Nb tags", int(row.TAGS_COUNT))
                tags = row.TAGS
                if tags is None:
                    tags = ""
                page.rich_text("Tags", tags)
                page.number("Nb emojis", int(row.EMOJIS_COUNT))
                emojis = row.EMOJIS
                if emojis is None:
                    emojis = ""
                page.rich_text("Emojis", emojis)
                page.number("Nb links", int(row.LINKS_COUNT))
                page.link("Links", row.LINKS)
                page.number("Nb characters", int(row.CHARACTER_COUNT))
                page.link("Content URL", post_url)
                
                # Page blocks
                text = row.TEXT
                split_text = text.split("\n")
                for t in split_text:
                    page.paragraph(t)

                # Add post content info
                content_title = row.CONTENT_TITLE
                if content_title is not None:
                    page.paragraph("\n")
                    page.paragraph("Content title:")
                    page.paragraph(content_title)
                content_url = row.CONTENT_URL
                if content_url is not None:
                    page.paragraph("Content url:")
                    page.paragraph(content_url)
                page.update()
                print(f"✅ Page '{title}' created in Notion.")
                
                # Page properties : dynamic
                update_dynamic_properties(page, row)
            else:
                # Page properties : dynamic
                update_dynamic_properties(page, row)            
        except Exception as e:
            print(f"❌ Error creating page '{title}' in Notion", e)
            print(row)

### Create new post page in Notion

In [ ]:
update_content_notion(df_posts_feed, notion_database)

### Update post stats in Notion

In [ ]:
update_content_notion(df_posts_stats, notion_database)

### Archive data in naas

In [ ]:
def archive_csv(df, filename="LINKEDIN_POSTS_FEED.csv", output_dir=None):
    # Add timestamp to df
    df["DATE_EXTRACT"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Get archive
    df_archive = pd.DataFrame()
    files = os.listdir(output_dir)
    for f in files:
        if f == filename:
            df_archive = pd.read_csv(filename)
            break
            
    # Concat
    df = pd.concat([df, df_archive], axis=0).drop_duplicates("POST_URN", keep="first")
    df.to_csv(filename, index=False)
    return df

In [ ]:
df = archive_csv(df_posts_feed)
print("💾 Linkedin posts archives:", len(df))